In [2]:
# Import libraries
import pandas as pd
import json
from pymongo import MongoClient
import mysql.connector
import time
from sqlalchemy import create_engine,text
from datetime import date
import random
from datetime import datetime

In [3]:
df_answers = pd.read_csv("dataset/Answers.csv", encoding="ISO-8859-1").dropna()
df_questions = pd.read_csv("dataset/Questions.csv", encoding="ISO-8859-1").dropna()
df_taglink = pd.read_csv("dataset/Tags.csv", encoding="ISO-8859-1").dropna()
df_tags = df_taglink.drop_duplicates(subset=['Tag'], keep='first').drop(columns=['Id'])
df_tags.head()

,Tag
0,python
1,osx
2,fonts
3,photoshop
5,windows


In [4]:
print(f'df_questions')
print({df_questions.info()})
print("\n------------------------------------------\n")
print(f'df_answers')
print(f'{df_answers.info()}')
print("\n------------------------------------------\n")
print(f'df_tags')
print(f'{df_tags.info()}')
print("\n------------------------------------------\n")
print(f'df_taglink')
print(f'{df_taglink.info()}')

df_questions
<class 'pandas.core.frame.DataFrame'>
Index: 601070 entries, 0 to 607281
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            601070 non-null  int64  
 1   OwnerUserId   601070 non-null  float64
 2   CreationDate  601070 non-null  object 
 3   Score         601070 non-null  int64  
 4   Title         601070 non-null  object 
 5   Body          601070 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 32.1+ MB
{None}

------------------------------------------

df_answers
<class 'pandas.core.frame.DataFrame'>
Index: 981755 entries, 0 to 987121
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            981755 non-null  int64  
 1   OwnerUserId   981755 non-null  float64
 2   CreationDate  981755 non-null  object 
 3   ParentId      981755 non-null  int64  
 4   Score         981755 non-null 

In [5]:
df_questions['OwnerUserId'] = df_questions['OwnerUserId'].apply(int)
df_questions['Title'] = df_questions['Title'].astype('string')
df_questions['Body'] = df_questions['Body'].astype('string')
df_answers['CreationDate'] = pd.to_datetime(df_answers['CreationDate'])
df_questions['CreationDate'] = pd.to_datetime(df_questions['CreationDate'])

In [6]:
print("Tables before filtering:")
print(df_questions.shape)
print(df_answers.shape)
print(df_tags.shape)
print(df_taglink.shape)

df_questions = df_questions[(df_questions['CreationDate'].dt.year == 2015) & (df_questions['CreationDate'].dt.month >= 9)]
df_answers= df_answers[(df_answers['CreationDate'].dt.year == 2015) & (df_answers['CreationDate'].dt.month >= 9)]
df_taglink = df_taglink[df_taglink['Id'].isin(df_questions['Id'])]

print("\nTables after filtering:")
print(df_questions.shape)
print(df_answers.shape)
print(df_tags.shape)
print(df_taglink.shape)

Tables before filtering:
(601070, 6)
(981755, 6)
(16895, 1)
(1884635, 2)

Tables after filtering:
(46722, 6)
(65576, 6)
(16895, 1)
(146233, 2)


In [7]:
df_questions['answers'] = df_questions['Id'].apply(lambda x: list(df_answers[df_answers['ParentId'] == x]['Id']))
df_questions.head()


,Id,OwnerUserId,CreationDate,Score,Title,Body,answers
429606,32321768,439834,2015-09-01 00:21:00+00:00,1,Python + Regex + Replace pattern with multiple...,<p>I have to take a string from the user and f...,[32322101]
429607,32321780,2954591,2015-09-01 00:22:25+00:00,1,"Why is this ""in"" statement not evaluating to T...",<p>I have this code here</p> <pre><code>input...,"[32321795, 32321809, 32321827]"
429608,32321888,4331195,2015-09-01 00:36:27+00:00,0,figure won't show in while loop,<p>as it will take a long time to deal with so...,"[32322020, 32322228]"
429609,32321902,2698521,2015-09-01 00:38:18+00:00,0,How to strip/remove row number from variable i...,<p>I get the follow output:</p> <p><code>\n35...,[32322297]
429610,32321906,211983,2015-09-01 00:38:47+00:00,0,attempting a post-deploy callback in fabric,<p>i use the following command to deploy to va...,[]


In [8]:
df_questions['tags'] = df_questions['Id'].apply(lambda x: list(df_taglink[df_taglink['Id'] == x]['Tag']))
df_questions.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body,answers,tags
429606,32321768,439834,2015-09-01 00:21:00+00:00,1,Python + Regex + Replace pattern with multiple...,<p>I have to take a string from the user and f...,[32322101],"[python, regex]"
429607,32321780,2954591,2015-09-01 00:22:25+00:00,1,"Why is this ""in"" statement not evaluating to T...",<p>I have this code here</p> <pre><code>input...,"[32321795, 32321809, 32321827]","[python, arrays, if-statement]"
429608,32321888,4331195,2015-09-01 00:36:27+00:00,0,figure won't show in while loop,<p>as it will take a long time to deal with so...,"[32322020, 32322228]","[python, plot, while-loop]"
429609,32321902,2698521,2015-09-01 00:38:18+00:00,0,How to strip/remove row number from variable i...,<p>I get the follow output:</p> <p><code>\n35...,[32322297],"[python, csv, datetime, row]"
429610,32321906,211983,2015-09-01 00:38:47+00:00,0,attempting a post-deploy callback in fabric,<p>i use the following command to deploy to va...,[],"[python, fabric]"


In [9]:
# Setting MongoDB database

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017")

# Drop and create database
client.drop_database("stackoverflow")
db = client['stackoverflow']

# Drop and create collections
db['Questions'].drop()
db['Answers'].drop()
db['Tags'].drop()
db['TagLink'].drop()
collection_questions = db['Questions']
collection_answers = db['Answers']
collection_tags = db['Tags']
collection_taglink = db['TagLink']

# Function to insert data into MongoDB
def insert_into_mongo(df, collection):
    initial_count = collection.count_documents({})
    print(f"# documents: {initial_count}") 
    try:
        data_dict = df.to_dict(orient="records")
        collection.insert_many(data_dict)
        valid_count = collection.count_documents({}) - initial_count
        print(f"{valid_count} documents inserted.")
    except Exception as e:
        attempted_count = len(data_dict)
        valid_count = collection.count_documents({}) - initial_count
        print(f"{valid_count} documents inserted")
        print(f"{attempted_count - valid_count} documents failed to insert")

insert_into_mongo(df_questions, collection_questions)
insert_into_mongo(df_answers, collection_answers)
insert_into_mongo(df_taglink, collection_taglink)
insert_into_mongo(df_tags, collection_tags)

# documents: 0
46722 documents inserted.
# documents: 0
65576 documents inserted.
# documents: 0
146233 documents inserted.
# documents: 0
16895 documents inserted.


In [10]:
queries = [
    {"description": "Retrieve all questions that have a score greater than 20", "query": ""},
    
    {"description": "Retrieve all answers from October of 2015", "query": ""},
    
    {"description": "Delete first tag on tags array in first question from 2015 that has more than 2 tags", "query": ""},

    {"description": "Insert answer for the most recent question from 2015 about PDF that have more than 4 answers", "query": ""}

]

In [12]:
#show how many data are on each month in answers dataframe of 2015
df_questions['CreationDate'].dt.month.value_counts()



CreationDate
10    12394
11    12348
12    11518
9     10462
Name: count, dtype: int64

In [19]:
df_taglink2 = pd.read_csv("dataset/Tags.csv", encoding="ISO-8859-1", nrows=500).dropna()

df_taglink2.shape

(500, 2)

In [13]:
from bson.objectid import ObjectId
mongo_times = []
for q in queries:
    start_time = time.time()
    
    if "score greater" in q["description"]:
        query = {'Score': {'$gt': 20}}
        result = collection_questions.find(query)
        count = collection_questions.count_documents(query)

    elif "of 2015" in q["description"]:
        query = {
            'CreationDate': {
                '$gte': datetime(2015, 10, 1),
                '$lte': datetime(2015, 10, 31)
            }
        }
        result = collection_answers.find(query)
        count = collection_answers.count_documents(query)

    elif "2 tags" in q["description"]:
        query = {
            'CreationDate': {
                '$gte': datetime(2015, 1, 1),
                '$lte': datetime(2015, 12, 31)
            },
            '$expr': {'$gte': [{'$size': '$tags'}, 2]}
        }
        result = collection_questions.find_one(query, sort=[('CreationDate', 1)])
        if result:
            collection_questions.update_one(
                {'_id': result['_id']},
                {'$pop': {'tags': -1}}
            )
            collection_taglink.delete_one({'Id': result['Id']})
            print(f"The deleted tag is: {result['tags'][-1]}, the question ID is: {result['_id']}")

        count = collection_questions.count_documents(query)
    else:
        query = {'CreationDate': {
                '$gte': datetime(2015, 1, 1),
                '$lte': datetime(2015, 12, 31)
            }, 
            'Title': {'$regex' : '.*pdf.*'}, 
            '$expr': {'$gte': [{'$size': '$answers'}, 4]}
            }
        result = collection_questions.find_one(query, sort=[('CreationDate', 1)])
        if result:
            last_id = collection_answers.find_one(sort=[('Id', -1)])['Id']
            awnser_id = ObjectId()
            parent_id = result['Id']
            parent_object_id = result['_id']
            new_answer = {
                '_id': awnser_id,
                'AnswerText': 'This is the new answer for this question.', 
                'CreationDate': datetime.now(),
                'OwnerUserId': random.randint(1, 1000),
                'ParentId': parent_id,
                'Score': 0,
                'Id': last_id + 1,
            }
            answer_result = collection_answers.insert_one(new_answer)
            answer_id = answer_result.inserted_id
            collection_questions.update_one({'_id': parent_object_id}, {'$push': {'answers': last_id+1}})
            print(f"Inserted new answer for question ID: {parent_object_id}, Answer ID: {answer_id}")
        count = collection_questions.count_documents(query)
    end_time = time.time()
    mongo_times.append(end_time - start_time)
    print(f"{q['description']}: {count} found in {end_time - start_time:.4f} seconds")

Retrieve all questions that have a score greater than 20: 57 found in 0.0265 seconds
Retrieve all answers from October of 2015: 17178 found in 0.0300 seconds
The deleted tag is: regex, the question ID is: 6749e74c5d62097d450e3f05
Delete first tag on tags array in first question from 2015 that has more than 2 tags: 43276 found in 0.1410 seconds
Inserted new answer for question ID: 6749e74c5d62097d450ec048, Answer ID: 6749e85f5d62097d451272e7
Insert answer for the most recent question from 2015 about PDF that have more than 4 answers: 2 found in 0.1940 seconds
